#### 1. Import necessary libraries

In [12]:
from bs4 import BeautifulSoup
import requests
import time, os, random
import re
import pandas as pd

In [40]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [41]:
driver = webdriver.Chrome(chromedriver)

/var/folders/4q/s5vk58v95472hjx15zb8y_940000gn/T/ipykernel_56759/2060176432.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


#### 2. Get total 23 pages of monitor listings on BestBuy.com

In [8]:
pages=['https://www.bestbuy.com/site/searchpage.jsp?id=pcat17071&st=monitor']
for i in range(2,24):
    new_url = 'https://www.bestbuy.com/site/searchpage.jsp?cp='+str(i)+'&id=pcat17071&st=monitor'
    pages.append(new_url)
pages[-3:]

['https://www.bestbuy.com/site/searchpage.jsp?cp=21&id=pcat17071&st=monitor',
 'https://www.bestbuy.com/site/searchpage.jsp?cp=22&id=pcat17071&st=monitor',
 'https://www.bestbuy.com/site/searchpage.jsp?cp=23&id=pcat17071&st=monitor']

#### 3. Iterate though pages to get each monitor link, product_name, brand, price, rating, review_count

In [9]:
main_page="https://www.bestbuy.com"
product_link=[]
product_name=[]
brand=[]
price=[]
rating=[]
review_count=[]

In [10]:
for page in pages:
    driver.get(page)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(2)
    
    ratings=[item.text.split(" ")[1] for item in soup.find(class_="sku-item-list")('p',class_="visually-hidden")]
    rating.extend(ratings)
    reviews=[item.text.split(" ")[-2] for item in soup.find(class_="sku-item-list")('p',class_="visually-hidden")]   
    review_count.extend(reviews)
    
    for ol in soup.find(class_="sku-item-list"):
        for li in ol.find_all('h4'):
            for link in li.find_all('a'):
                product_link.append(main_page+link['href'])
                product_name.append(link.text)
                brand.append(link.text.split(' ')[0])

        for price_ in ol.find_all('div', class_="price-block"):
            price.append(price_('span')[0].text.split(">$")[-1])       



In [132]:
len(set(product_link))

551

In [146]:
print({"product_link":len(product_link),
       "product_name":len(product_name),
       "brand":len(brand),
       "price":len(price),
       "rating":len(rating),
       "review_count":len(review_count)
    
      })

{'product_link': 574, 'product_name': 574, 'brand': 574, 'price': 574, 'rating': 574, 'review_count': 574}


In [11]:
import pandas as pd
df= pd.DataFrame(
    {'product_link': product_link,
     'product_name' : product_name,
     'brand':brand,
     'current_price':price,
     'rating': rating,
     'review_count':review_count
    })
df

,product_link,product_name,brand,current_price,rating,review_count
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840
...,...,...,...,...,...,...
569,https://www.bestbuy.com/site/apple-studio-disp...,Apple - Studio Display - Standard Glass Tilt a...,Apple,"$1,999.99",Yet,Yet
570,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet
571,https://www.bestbuy.com/site/dell-geek-squad-c...,Dell - Geek Squad Certified Refurbished UltraS...,Dell,"$3,599.99",Yet,Yet
572,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 23.8"" IP...",HP,$179.99,Yet,Yet


In [12]:
# A bit of data cleaning
df.product_link.nunique()

551

In [14]:
df.drop_duplicates(inplace=True)
df

,product_link,product_name,brand,current_price,rating,review_count
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840
...,...,...,...,...,...,...
569,https://www.bestbuy.com/site/apple-studio-disp...,Apple - Studio Display - Standard Glass Tilt a...,Apple,"$1,999.99",Yet,Yet
570,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet
571,https://www.bestbuy.com/site/dell-geek-squad-c...,Dell - Geek Squad Certified Refurbished UltraS...,Dell,"$3,599.99",Yet,Yet
572,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 23.8"" IP...",HP,$179.99,Yet,Yet


In [19]:
# Save the data to local 
import os  
df.to_csv('Monitor_info.csv',index=False)  

#### 4. Iterate through product links to extract monitor features

In [4]:
df=pd.read_csv('Monitor_info.csv')
df

,product_link,product_name,brand,current_price,rating,review_count
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840
...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/apple-studio-disp...,Apple - Studio Display - Standard Glass Tilt a...,Apple,"$1,999.99",Yet,Yet
547,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet
548,https://www.bestbuy.com/site/dell-geek-squad-c...,Dell - Geek Squad Certified Refurbished UltraS...,Dell,"$3,599.99",Yet,Yet
549,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 23.8"" IP...",HP,$179.99,Yet,Yet


In [5]:
list(df['product_link'])[0:3]

['https://www.bestbuy.com/site/samsung-t350-series-24-ips-led-fhd-freesync-hdmi-vga-dark-blue-gray/6420870.p?skuId=6420870',
 'https://www.bestbuy.com/site/samsung-390-series-24-led-curved-fhd-freesync-monitor-hdmi-vga-high-glossy-black/5044601.p?skuId=5044601',
 'https://www.bestbuy.com/site/lg-34-ultrawide-full-hd-hdr-monitor-with-freesync/6451082.p?skuId=6451082']

In [174]:
linklink=[]
Refresh_Rate=[]
Resolution=[]
Contrast_Ratio=[]
Response_Time=[]
Display_Type=[]
Panel_Type=[]
Synchronization_Technology=[]
Curved_Screen=[]
Pixel_Density=[]
Brightness=[]
Screen_Size=[]
HDR=[]
Smart_Display=[]
Touch_Screen=[]
Quantum_Dot_Technology=[]
Headphone_Jack=[]
Voice_Assistant=[]
Wall_Mountable=[]
Speaker=[]
Webcam=[]
Weight=[]
Power_Consumption=[]
Anti_Glare=[]

In [6]:
products=list(df['product_link'])[0:]

for product in products:
    index = products.index(product)
    driver.get(product)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1 + 2*random.random())
    print(index,product)
    linklink.append(product)
    
    #Refresh_Rate
    refresh=soup.find(class_="display-name v-fw-medium body-copy", text="Refresh Rate")
    if refresh==None:
        Refresh_Rate.append(None)
    if refresh!=None:
        Refresh_Rate.append(refresh.parent.parent.parent.nextSibling.text.strip("Hz") )
    
    #Resolution
    resolution=soup.find(class_="display-name v-fw-medium body-copy", text="Maximum Resolution")
    if resolution==None:
        Resolution.append(None)
    if resolution!=None:
        Resolution.append(resolution.parent.parent.parent.nextSibling.text)
    
    #Contrast Ratio
    contrast=soup.find(class_="display-name v-fw-medium body-copy", text="Contrast Ratio")
    if contrast==None:
        Contrast_Ratio.append(None)
    if contrast!=None:
        Contrast_Ratio.append(contrast.parent.parent.parent.nextSibling.text[:-2].replace(",",""))
    
    #Response Time
    find_response=soup.find(class_="display-name v-fw-medium body-copy", text="Response Time")
    if find_response==None:
        Response_Time.append(None)
    if find_response!=None:
        Response_Time.append(find_response.parent.parent.parent.nextSibling.text.strip(" milliseconds"))
        
    #Display Type
    display=soup.find(class_="row-title", text=re.compile("Display Type"))
    if display==None:
        Display_Type.append(None)
    if display!=None:
        Display_Type.append(display.parent.nextSibling.text)    
    
    #Panel Type
    panel=soup.find(class_="display-name v-fw-medium body-copy", text="Panel Type")
    if panel==None:
        Panel_Type.append(None)
    if panel!=None:
        Panel_Type.append(panel.parent.parent.parent.nextSibling.text)
        
    #Synchronization Technology
    synchronization=soup.find(class_="display-name v-fw-medium body-copy", text="Synchronization Technology")
    if synchronization==None:
        Synchronization_Technology.append(None)
    if synchronization!=None:
        Synchronization_Technology.append(synchronization.parent.parent.parent.nextSibling.text)
    
    #Curved_Screen
    curved=soup.find(class_="row-title", text=re.compile("Curved Screen"))
    if curved==None:
        Curved_Screen.append(None)
    if curved!=None:
        Curved_Screen.append(curved.parent.nextSibling.text)
    
    #Pixel Density
    pixel=soup.find(class_="display-name v-fw-medium body-copy", text="Pixel Density")
    if pixel==None:
        Pixel_Density.append(None)
    if pixel!=None:
        Pixel_Density.append(pixel.parent.parent.parent.nextSibling.text.strip(" pixels per inch"))

    #Brightness
    bright=soup.find(class_="display-name v-fw-medium body-copy", text="Brightness")
    if bright==None:
        Brightness.append(None)
    if bright!=None:
        Brightness.append(bright.parent.parent.parent.nextSibling.text.strip(" candela per square meter"))
                
    ##Screen Size
    size=soup.find(class_="display-name v-fw-medium body-copy", text="Screen Size")
    if size==None:
        Screen_Size.append(None)
    if size!=None:
        Screen_Size.append(size.parent.parent.parent.nextSibling.text.strip(" inches"))  
    
    
    #High Dynamic Range (HDR)
    hdr=soup.find(class_="display-name v-fw-medium body-copy", text="High Dynamic Range (HDR)")
    if hdr==None:
        HDR.append(None)
    if hdr!=None:
        HDR.append(hdr.parent.parent.parent.nextSibling.text)
        
    #Smart Display
    smart=soup.find(class_="display-name v-fw-medium body-copy", text="Smart Display")
    if smart==None:
        Smart_Display.append(None)
    if smart!=None:
        Smart_Display.append(smart.parent.parent.parent.nextSibling.text)

    #Touch Screen
    touch=soup.find(class_="display-name v-fw-medium body-copy", text="Touch Screen")
    if touch==None:
        Touch_Screen.append(None)
    if touch!=None:
        Touch_Screen.append(touch.parent.parent.parent.nextSibling.text)

    #Quantum_Dot_Technology
    quantum=soup.find(class_="display-name v-fw-medium body-copy", text="Quantum Dot Technology")
    if quantum==None:
        Quantum_Dot_Technology.append(None)
    if quantum!=None:
        Quantum_Dot_Technology.append(quantum.parent.parent.parent.nextSibling.text)

    #Headphone_Jack
    headphone=soup.find(class_="display-name v-fw-medium body-copy", text="Headphone Jack")
    if headphone==None:
        Headphone_Jack.append(None)
    if headphone!=None:
        Headphone_Jack.append(headphone.parent.parent.parent.nextSibling.text)

    #Voice Assistant Built-in
    voice=soup.find(class_="display-name v-fw-medium body-copy", text="Voice Assistant Built-in")
    if voice==None:
        Voice_Assistant.append(None)
    if voice!=None:
        Voice_Assistant.append(voice.parent.parent.parent.nextSibling.text)
        
    #Wall Mountable
    mount=soup.find(class_="display-name v-fw-medium body-copy", text="Wall Mountable")
    if mount==None:
        Wall_Mountable.append(None)
    if mount!=None:
        Wall_Mountable.append(mount.parent.parent.parent.nextSibling.text)

    #Integrated Speaker(s)
    speaker=soup.find(class_="row-title", text=re.compile("Speaker"))
    if speaker==None:
        Speaker.append(None)
    if speaker!=None:
        Speaker.append(speaker.parent.nextSibling.text)   

    #Built-In Webcam 
    power=soup.find(class_="row-title", text=re.compile("Power Consumption"))
    if power==None:
        Power_Consumption.append(None)
    if power!=None:
        Power_Consumption.append(power.parent.nextSibling.text[:-5])  
        
    #Weight
    weight=soup.find(class_="display-name v-fw-medium body-copy", text="Product Weight")
    if weight==None:
        Weight.append(None)
    if weight!=None:
        Weight.append(weight.parent.parent.parent.nextSibling.text.strip(" pounds"))
        
    #Power Consumption 
    webcam=soup.find(class_="row-title", text=re.compile("Webcam"))
    if webcam==None:
        Webcam.append(None)
    if webcam!=None:
        Webcam.append(webcam.parent.nextSibling.text)   
        
    #Anti_Glare
    glare=soup.find(class_="display-name v-fw-medium body-copy", text="Anti-Glare")
    if glare==None:
        Anti_Glare.append(None)
    if glare!=None:
        Anti_Glare.append(glare.parent.parent.parent.nextSibling.text)
        

In [ ]:
# print({"linklink":len(linklink),"Refresh_Rate":len(Refresh_Rate),"Resolution":len(Resolution),
#        "Contrast_Ratio":len(Contrast_Ratio),"Response_Time":len(Response_Time),
#        "Synchronization_Technology":len(Synchronization_Technology),"Panel_Type":len(Panel_Type), 
#        "Curved_Screen":len(Curved_Screen),
#        "Pixel_Density":len(Pixel_Density),"Brightness":len(Brightness),"Screen_Size":len(Screen_Size),
#        "HDR":len(HDR),"Smart_Display":len(Smart_Display),"Touch_Screen":len(Touch_Screen),
#        "Quantum_Dot_Technology":len(Quantum_Dot_Technology),"Headphone_Jack":len(Headphone_Jack),
#        "Voice_Assistant":len(Voice_Assistant),"Wall_Mountable":len(Wall_Mountable),"Anti_Glare":len(Anti_Glare),
#        "Speaker":len(Speaker),"Webcam":len(Webcam),"Power_Consumption":len(Power_Consumption),"Weight":len(Weight),
#        "Anti_Glare":len(Anti_Glare)
      
#       })

In [231]:
import pandas as pd
features= pd.DataFrame(
    {'linklink': linklink,
     'Refresh_Rate' : Refresh_Rate,
     'Contrast_Ratio':Contrast_Ratio,
     'Response_Time':Response_Time,
     'Synchronization_Technology': Synchronization_Technology,
     'Panel_Type':Panel_Type,
     'Curved_Screen': Curved_Screen,
     'Pixel_Density' : Pixel_Density,
     'Contrast_Ratio':Contrast_Ratio,
     'Brightness':Brightness,
     'Screen_Size': Screen_Size,
     'HDR':HDR,
     'Smart_Display':Smart_Display,
     'Touch_Screen':Touch_Screen,
     'Quantum_Dot_Technology':Quantum_Dot_Technology,
     'Headphone_Jack':Headphone_Jack,
     'Voice_Assistant':Voice_Assistant,
     'Wall_Mountable':Wall_Mountable,
     'Anti_Glare':Anti_Glare,
     'Speaker':Speaker,
     'Webcam':Webcam,
     'Power_Consumption':Power_Consumption,
     'Weight':Weight     
    })
features

,linklink,Refresh_Rate,Contrast_Ratio,Response_Time,Synchronization_Technology,Panel_Type,Curved_Screen,Pixel_Density,Brightness,Screen_Size,...,Touch_Screen,Quantum_Dot_Technology,Headphone_Jack,Voice_Assistant,Wall_Mountable,Anti_Glare,Speaker,Webcam,Power_Consumption,Weight
0,https://www.bestbuy.com/site/samsung-t350-seri...,75,1000,5,FreeSync (AMD Adaptive Sync),IPS,No,92,250,24,...,No,No,No,No,Yes,None,No,No,35,5.95
1,https://www.bestbuy.com/site/samsung-390-serie...,60,3000,4,FreeSync (AMD Adaptive Sync),VA,Yes,None,250,24,...,No,None,Yes,No,Yes,No,No,No,25,7.3
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,75,1000,5,FreeSync (AMD Adaptive Sync),IPS,No,0,250,34,...,No,No,Yes,Not Applicable,Yes,Yes,No,No,39,13.1
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,165,3000,2,FreeSync Premium (AMD Adaptive Sync),VA,Yes,93,350,32,...,No,No,Yes,Not Applicable,Yes,Yes,No,No,19.9,22.4
4,https://www.bestbuy.com/site/samsung-ur55-seri...,60,1000,4,FreeSync (AMD Adaptive Sync),IPS,No,157,300,28,...,No,No,No,No,Yes,None,No,No,40,12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/apple-studio-disp...,60,1000,None,Not Applicable,Retina Display,No,None,600,27,...,None,None,None,None,No,None,Yes,Yes,None,16.9
547,https://www.bestbuy.com/site/dell-geek-squad-c...,60,1300,4,FreeSync (AMD Adaptive Sync),IPS,No,None,350,27,...,No,None,Yes,Not Applicable,Yes,Yes,Yes,No,28,14.1
548,https://www.bestbuy.com/site/dell-geek-squad-c...,60,1000000,5,None,IPS,No,163,350,31.5,...,No,None,No,None,Yes,Yes,No,No,70,38.1
549,https://www.bestbuy.com/site/hp-geek-squad-cer...,75,10000000,5,FreeSync (AMD Adaptive Sync),IPS,No,93,300,23.8,...,None,None,None,None,None,Yes,None,No,20,6.3


In [248]:
# Save the data to local 
features.to_csv('Monitor_features.csv',index=False) 

#### 5. Merge dataframes

In [272]:
info=pd.read_csv("Monitor_info.csv")
features=pd.read_csv("Monitor_features.csv")

In [273]:
info

,product_link,product_name,brand,current_price,rating,review_count
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840
...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/apple-studio-disp...,Apple - Studio Display - Standard Glass Tilt a...,Apple,"$1,999.99",Yet,Yet
547,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet
548,https://www.bestbuy.com/site/dell-geek-squad-c...,Dell - Geek Squad Certified Refurbished UltraS...,Dell,"$3,599.99",Yet,Yet
549,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 23.8"" IP...",HP,$179.99,Yet,Yet


In [274]:
features

,linklink,Refresh_Rate,Contrast_Ratio,Response_Time,Synchronization_Technology,Panel_Type,Curved_Screen,Pixel_Density,Brightness,Screen_Size,...,Touch_Screen,Quantum_Dot_Technology,Headphone_Jack,Voice_Assistant,Wall_Mountable,Anti_Glare,Speaker,Webcam,Power_Consumption,Weight
0,https://www.bestbuy.com/site/samsung-t350-seri...,75,1000,5,FreeSync (AMD Adaptive Sync),IPS,No,92,250,24,...,No,No,No,No,Yes,None,No,No,35,5.95
1,https://www.bestbuy.com/site/samsung-390-serie...,60,3000,4,FreeSync (AMD Adaptive Sync),VA,Yes,None,250,24,...,No,None,Yes,No,Yes,No,No,No,25,7.3
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,75,1000,5,FreeSync (AMD Adaptive Sync),IPS,No,0,250,34,...,No,No,Yes,Not Applicable,Yes,Yes,No,No,39,13.1
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,165,3000,2,FreeSync Premium (AMD Adaptive Sync),VA,Yes,93,350,32,...,No,No,Yes,Not Applicable,Yes,Yes,No,No,19.9,22.4
4,https://www.bestbuy.com/site/samsung-ur55-seri...,60,1000,4,FreeSync (AMD Adaptive Sync),IPS,No,157,300,28,...,No,No,No,No,Yes,None,No,No,40,12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/apple-studio-disp...,60,1000,None,Not Applicable,Retina Display,No,None,600,27,...,None,None,None,None,No,None,Yes,Yes,None,16.9
547,https://www.bestbuy.com/site/dell-geek-squad-c...,60,1300,4,FreeSync (AMD Adaptive Sync),IPS,No,None,350,27,...,No,None,Yes,Not Applicable,Yes,Yes,Yes,No,28,14.1
548,https://www.bestbuy.com/site/dell-geek-squad-c...,60,1000000,5,None,IPS,No,163,350,31.5,...,No,None,No,None,Yes,Yes,No,No,70,38.1
549,https://www.bestbuy.com/site/hp-geek-squad-cer...,75,10000000,5,FreeSync (AMD Adaptive Sync),IPS,No,93,300,23.8,...,None,None,None,None,None,Yes,None,No,20,6.3


In [285]:
monitor=pd.merge(info, features, left_on="product_link",right_on="linklink")
monitor.drop(columns=['linklink'],inplace=True)

In [284]:
monitor

,product_link,product_name,brand,current_price,rating,review_count,Refresh_Rate,Contrast_Ratio,Response_Time,Synchronization_Technology,...,Touch_Screen,Quantum_Dot_Technology,Headphone_Jack,Voice_Assistant,Wall_Mountable,Anti_Glare,Speaker,Webcam,Power_Consumption,Weight
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838,75,1000,5,FreeSync (AMD Adaptive Sync),...,No,No,No,No,Yes,None,No,No,35,5.95
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106,60,3000,4,FreeSync (AMD Adaptive Sync),...,No,None,Yes,No,Yes,No,No,No,25,7.3
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425,75,1000,5,FreeSync (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,39,13.1
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277,165,3000,2,FreeSync Premium (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,19.9,22.4
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840,60,1000,4,FreeSync (AMD Adaptive Sync),...,No,No,No,No,Yes,None,No,No,40,12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/apple-studio-disp...,Apple - Studio Display - Standard Glass Tilt a...,Apple,"$1,999.99",Yet,Yet,60,1000,None,Not Applicable,...,None,None,None,None,No,None,Yes,Yes,None,16.9
547,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet,60,1300,4,FreeSync (AMD Adaptive Sync),...,No,None,Yes,Not Applicable,Yes,Yes,Yes,No,28,14.1
548,https://www.bestbuy.com/site/dell-geek-squad-c...,Dell - Geek Squad Certified Refurbished UltraS...,Dell,"$3,599.99",Yet,Yet,60,1000000,5,None,...,No,None,No,None,Yes,Yes,No,No,70,38.1
549,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 23.8"" IP...",HP,$179.99,Yet,Yet,75,10000000,5,FreeSync (AMD Adaptive Sync),...,None,None,None,None,None,Yes,None,No,20,6.3


In [281]:
# Save the data to local 
monitor.to_csv('Monitor.csv',index=False) 

### The above data scraping was made on May 14, Saturday. Given that my target is the real time prices of monitors on BestBuy, monitor listings and prices are likely to change over time, I repeated the scraping for the date May 17, Tuesday. Other features such as rating, review_count are likely to change as well.

In [18]:
pages=['https://www.bestbuy.com/site/searchpage.jsp?id=pcat17071&st=monitor']
for i in range(2,24):
    new_url = 'https://www.bestbuy.com/site/searchpage.jsp?cp='+str(i)+'&id=pcat17071&st=monitor'
    pages.append(new_url)
pages[-3:]

['https://www.bestbuy.com/site/searchpage.jsp?cp=21&id=pcat17071&st=monitor',
 'https://www.bestbuy.com/site/searchpage.jsp?cp=22&id=pcat17071&st=monitor',
 'https://www.bestbuy.com/site/searchpage.jsp?cp=23&id=pcat17071&st=monitor']

In [19]:
main_page="https://www.bestbuy.com"
product_link=[]
product_name=[]
brand=[]
price=[]
rating=[]
review_count=[]

In [20]:
for page in pages:
    driver.get(page)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(2)
    
    ratings=[item.text.split(" ")[1] for item in soup.find(class_="sku-item-list")('p',class_="visually-hidden")]
    rating.extend(ratings)
    reviews=[item.text.split(" ")[-2] for item in soup.find(class_="sku-item-list")('p',class_="visually-hidden")]   
    review_count.extend(reviews)
    
    for ol in soup.find(class_="sku-item-list"):
        for li in ol.find_all('h4'):
            for link in li.find_all('a'):
                product_link.append(main_page+link['href'])
                product_name.append(link.text)
                brand.append(link.text.split(' ')[0])

        for price_ in ol.find_all('div', class_="price-block"):
            price.append(price_('span')[0].text.split(">$")[-1])       

In [22]:
import pandas as pd
df1= pd.DataFrame(
    {'product_link': product_link,
     'product_name' : product_name,
     'brand':brand,
     'current_price':price,
     'rating': rating,
     'review_count':review_count
    })
df1.drop_duplicates(inplace=True)
df1

,product_link,product_name,brand,current_price,rating,review_count
0,https://www.bestbuy.com/site/viewsonic-va2447-...,ViewSonic VA2447-MH 24 Inch Full HD 1080p Moni...,ViewSonic,$139.99,Yet,Yet
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5111
2,https://www.bestbuy.com/site/lg-24-ips-led-fhd...,"LG - 24"" IPS LED FHD FreeSync Monitor (HDMI, V...",LG,$129.99,4.6,2728
3,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$329.99,4.5,433
4,https://www.bestbuy.com/site/hp-24-ips-led-fhd...,"HP - 24"" IPS LED FHD FreeSync Monitor (HDMI, V...",HP,$149.99,4.7,629
...,...,...,...,...,...,...
570,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet
571,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet
572,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 27"" IPS ...",HP,$249.99,Yet,Yet
573,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet


In [23]:
# Save the data to local   
df1.to_csv('Monitor_info_1.csv',index=False)  

In [26]:
df1=pd.read_csv('Monitor_info_1.csv')
df1

,product_link,product_name,brand,current_price,rating,review_count
0,https://www.bestbuy.com/site/viewsonic-va2447-...,ViewSonic VA2447-MH 24 Inch Full HD 1080p Moni...,ViewSonic,$139.99,Yet,Yet
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5111
2,https://www.bestbuy.com/site/lg-24-ips-led-fhd...,"LG - 24"" IPS LED FHD FreeSync Monitor (HDMI, V...",LG,$129.99,4.6,2728
3,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$329.99,4.5,433
4,https://www.bestbuy.com/site/hp-24-ips-led-fhd...,"HP - 24"" IPS LED FHD FreeSync Monitor (HDMI, V...",HP,$149.99,4.7,629
...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet
547,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet
548,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 27"" IPS ...",HP,$249.99,Yet,Yet
549,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet


In [ ]:
linklink=[]
Refresh_Rate=[]
Resolution=[]
Contrast_Ratio=[]
Response_Time=[]
Display_Type=[]
Panel_Type=[]
Synchronization_Technology=[]
Curved_Screen=[]
Pixel_Density=[]
Brightness=[]
Screen_Size=[]
HDR=[]
Smart_Display=[]
Touch_Screen=[]
Quantum_Dot_Technology=[]
Headphone_Jack=[]
Voice_Assistant=[]
Wall_Mountable=[]
Speaker=[]
Webcam=[]
Weight=[]
Power_Consumption=[]
Anti_Glare=[]

In [106]:
products=list(df1['product_link'])[0:]

for product in products:
    driver.get(product)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1 + 2*random.random())
    linklink.append(product)
    
    #Refresh_Rate
    refresh=soup.find(class_="display-name v-fw-medium body-copy", text="Refresh Rate")
    if refresh==None:
        Refresh_Rate.append(None)
    if refresh!=None:
        Refresh_Rate.append(refresh.parent.parent.parent.nextSibling.text.strip("Hz") )
    
    #Resolution
    resolution=soup.find(class_="display-name v-fw-medium body-copy", text="Maximum Resolution")
    if resolution==None:
        Resolution.append(None)
    if resolution!=None:
        Resolution.append(resolution.parent.parent.parent.nextSibling.text)
    
    #Contrast Ratio
    contrast=soup.find(class_="display-name v-fw-medium body-copy", text="Contrast Ratio")
    if contrast==None:
        Contrast_Ratio.append(None)
    if contrast!=None:
        Contrast_Ratio.append(contrast.parent.parent.parent.nextSibling.text[:-2].replace(",",""))
    
    #Response Time
    find_response=soup.find(class_="display-name v-fw-medium body-copy", text="Response Time")
    if find_response==None:
        Response_Time.append(None)
    if find_response!=None:
        Response_Time.append(find_response.parent.parent.parent.nextSibling.text.strip(" milliseconds"))
        
    #Display Type
    display=soup.find(class_="row-title", text=re.compile("Display Type"))
    if display==None:
        Display_Type.append(None)
    if display!=None:
        Display_Type.append(display.parent.nextSibling.text)    
    
    #Panel Type
    panel=soup.find(class_="display-name v-fw-medium body-copy", text="Panel Type")
    if panel==None:
        Panel_Type.append(None)
    if panel!=None:
        Panel_Type.append(panel.parent.parent.parent.nextSibling.text)
        
    #Synchronization Technology
    synchronization=soup.find(class_="display-name v-fw-medium body-copy", text="Synchronization Technology")
    if synchronization==None:
        Synchronization_Technology.append(None)
    if synchronization!=None:
        Synchronization_Technology.append(synchronization.parent.parent.parent.nextSibling.text)
    
    #Curved_Screen
    curved=soup.find(class_="row-title", text=re.compile("Curved Screen"))
    if curved==None:
        Curved_Screen.append(None)
    if curved!=None:
        Curved_Screen.append(curved.parent.nextSibling.text)
    
    #Pixel Density
    pixel=soup.find(class_="display-name v-fw-medium body-copy", text="Pixel Density")
    if pixel==None:
        Pixel_Density.append(None)
    if pixel!=None:
        Pixel_Density.append(pixel.parent.parent.parent.nextSibling.text.strip(" pixels per inch"))

    #Brightness
    bright=soup.find(class_="display-name v-fw-medium body-copy", text="Brightness")
    if bright==None:
        Brightness.append(None)
    if bright!=None:
        Brightness.append(bright.parent.parent.parent.nextSibling.text.strip(" candela per square meter"))
                
    ##Screen Size
    size=soup.find(class_="display-name v-fw-medium body-copy", text="Screen Size")
    if size==None:
        Screen_Size.append(None)
    if size!=None:
        Screen_Size.append(size.parent.parent.parent.nextSibling.text.strip(" inches"))  
    
    
    #High Dynamic Range (HDR)
    hdr=soup.find(class_="display-name v-fw-medium body-copy", text="High Dynamic Range (HDR)")
    if hdr==None:
        HDR.append(None)
    if hdr!=None:
        HDR.append(hdr.parent.parent.parent.nextSibling.text)
        
    #Smart Display
    smart=soup.find(class_="display-name v-fw-medium body-copy", text="Smart Display")
    if smart==None:
        Smart_Display.append(None)
    if smart!=None:
        Smart_Display.append(smart.parent.parent.parent.nextSibling.text)

    #Touch Screen
    touch=soup.find(class_="display-name v-fw-medium body-copy", text="Touch Screen")
    if touch==None:
        Touch_Screen.append(None)
    if touch!=None:
        Touch_Screen.append(touch.parent.parent.parent.nextSibling.text)

    #Quantum_Dot_Technology
    quantum=soup.find(class_="display-name v-fw-medium body-copy", text="Quantum Dot Technology")
    if quantum==None:
        Quantum_Dot_Technology.append(None)
    if quantum!=None:
        Quantum_Dot_Technology.append(quantum.parent.parent.parent.nextSibling.text)

    #Headphone_Jack
    headphone=soup.find(class_="display-name v-fw-medium body-copy", text="Headphone Jack")
    if headphone==None:
        Headphone_Jack.append(None)
    if headphone!=None:
        Headphone_Jack.append(headphone.parent.parent.parent.nextSibling.text)

    #Voice Assistant Built-in
    voice=soup.find(class_="display-name v-fw-medium body-copy", text="Voice Assistant Built-in")
    if voice==None:
        Voice_Assistant.append(None)
    if voice!=None:
        Voice_Assistant.append(voice.parent.parent.parent.nextSibling.text)
        
    #Wall Mountable
    mount=soup.find(class_="display-name v-fw-medium body-copy", text="Wall Mountable")
    if mount==None:
        Wall_Mountable.append(None)
    if mount!=None:
        Wall_Mountable.append(mount.parent.parent.parent.nextSibling.text)

    #Integrated Speaker(s)
    speaker=soup.find(class_="row-title", text=re.compile("Speaker"))
    if speaker==None:
        Speaker.append(None)
    if speaker!=None:
        Speaker.append(speaker.parent.nextSibling.text)   

    #Built-In Webcam 
    power=soup.find(class_="row-title", text=re.compile("Power Consumption"))
    if power==None:
        Power_Consumption.append(None)
    if power!=None:
        Power_Consumption.append(power.parent.nextSibling.text[:-5])  
        
    #Weight
    weight=soup.find(class_="display-name v-fw-medium body-copy", text="Product Weight")
    if weight==None:
        Weight.append(None)
    if weight!=None:
        Weight.append(weight.parent.parent.parent.nextSibling.text.strip(" pounds"))
        
    #Power Consumption 
    webcam=soup.find(class_="row-title", text=re.compile("Webcam"))
    if webcam==None:
        Webcam.append(None)
    if webcam!=None:
        Webcam.append(webcam.parent.nextSibling.text)   
        
    #Anti_Glare
    glare=soup.find(class_="display-name v-fw-medium body-copy", text="Anti-Glare")
    if glare==None:
        Anti_Glare.append(None)
    if glare!=None:
        Anti_Glare.append(glare.parent.parent.parent.nextSibling.text)
        

In [108]:
features_1= pd.DataFrame(
    {'linklink': linklink,
     'Refresh_Rate' : Refresh_Rate,
     'Contrast_Ratio':Contrast_Ratio,
     'Response_Time':Response_Time,
     'Synchronization_Technology': Synchronization_Technology,
     'Panel_Type':Panel_Type,
     'Curved_Screen': Curved_Screen,
     'Pixel_Density' : Pixel_Density,
     'Contrast_Ratio':Contrast_Ratio,
     'Brightness':Brightness,
     'Screen_Size': Screen_Size,
     'HDR':HDR,
     'Smart_Display':Smart_Display,
     'Touch_Screen':Touch_Screen,
     'Quantum_Dot_Technology':Quantum_Dot_Technology,
     'Headphone_Jack':Headphone_Jack,
     'Voice_Assistant':Voice_Assistant,
     'Wall_Mountable':Wall_Mountable,
     'Anti_Glare':Anti_Glare,
     'Speaker':Speaker,
     'Webcam':Webcam,
     'Power_Consumption':Power_Consumption,
     'Weight':Weight     
    })
features_1.shape

(551, 22)

In [109]:
# Save the data to local 
features_1.to_csv('Monitor_features_1.csv',index=False) 

In [110]:
info_1=pd.read_csv("Monitor_info_1.csv")
features_1=pd.read_csv("Monitor_features_1.csv")

In [112]:
monitor_1=pd.merge(info_1, features_1, left_on="product_link",right_on="linklink")
monitor_1.drop(columns=['linklink'],inplace=True)

In [113]:
# Save the data to local 
monitor_1.to_csv('Monitor_1.csv',index=False) 

In [132]:
monitor_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 0 to 550
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_link                551 non-null    object 
 1   product_name                551 non-null    object 
 2   brand                       551 non-null    object 
 3   current_price               551 non-null    object 
 4   rating                      551 non-null    object 
 5   review_count                551 non-null    object 
 6   Refresh_Rate                547 non-null    object 
 7   Contrast_Ratio              547 non-null    object 
 8   Response_Time               492 non-null    float64
 9   Synchronization_Technology  529 non-null    object 
 10  Panel_Type                  544 non-null    object 
 11  Curved_Screen               535 non-null    object 
 12  Pixel_Density               90 non-null     float64
 13  Brightness                  547 non

### Final Data

In [116]:
monitor=pd.read_csv('Monitor.csv')

In [130]:
monitor  # Data for May 14

,product_link,product_name,brand,current_price,rating,review_count,Refresh_Rate,Contrast_Ratio,Response_Time,Synchronization_Technology,...,Touch_Screen,Quantum_Dot_Technology,Headphone_Jack,Voice_Assistant,Wall_Mountable,Anti_Glare,Speaker,Webcam,Power_Consumption,Weight
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838,75,1000,5.0,FreeSync (AMD Adaptive Sync),...,No,No,No,No,Yes,NaN,No,No,35.0,5.95
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106,60,3000,4.0,FreeSync (AMD Adaptive Sync),...,No,NaN,Yes,No,Yes,No,No,No,25.0,7.30
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425,75,1000,5.0,FreeSync (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,39.0,13.10
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277,165,3000,2.0,FreeSync Premium (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,19.9,22.40
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840,60,1000,4.0,FreeSync (AMD Adaptive Sync),...,No,No,No,No,Yes,NaN,No,No,40.0,12.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/apple-studio-disp...,Apple - Studio Display - Standard Glass Tilt a...,Apple,"$1,999.99",Yet,Yet,60,1000,NaN,Not Applicable,...,NaN,NaN,NaN,NaN,No,NaN,Yes,Yes,NaN,16.90
547,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet,60,1300,4.0,FreeSync (AMD Adaptive Sync),...,No,NaN,Yes,Not Applicable,Yes,Yes,Yes,No,28.0,14.10
548,https://www.bestbuy.com/site/dell-geek-squad-c...,Dell - Geek Squad Certified Refurbished UltraS...,Dell,"$3,599.99",Yet,Yet,60,1000000,5.0,NaN,...,No,NaN,No,NaN,Yes,Yes,No,No,70.0,38.10
549,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 23.8"" IP...",HP,$179.99,Yet,Yet,75,10000000,5.0,FreeSync (AMD Adaptive Sync),...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,No,20.0,6.30


In [119]:
monitor_1   # Data for May 17

,product_link,product_name,brand,current_price,rating,review_count,Refresh_Rate,Contrast_Ratio,Response_Time,Synchronization_Technology,...,Touch_Screen,Quantum_Dot_Technology,Headphone_Jack,Voice_Assistant,Wall_Mountable,Anti_Glare,Speaker,Webcam,Power_Consumption,Weight
0,https://www.bestbuy.com/site/viewsonic-va2447-...,ViewSonic VA2447-MH 24 Inch Full HD 1080p Moni...,ViewSonic,$139.99,Yet,Yet,75,1000,5.0,Not Applicable,...,No,NaN,Yes,NaN,Yes,Yes,Yes,No,23.0,7.3
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5111,60,3000,4.0,FreeSync (AMD Adaptive Sync),...,No,NaN,Yes,No,Yes,No,No,No,25.0,7.3
2,https://www.bestbuy.com/site/lg-24-ips-led-fhd...,"LG - 24"" IPS LED FHD FreeSync Monitor (HDMI, V...",LG,$129.99,4.6,2728,75,1000,5.0,FreeSync (AMD Adaptive Sync),...,NaN,NaN,Yes,NaN,Yes,NaN,No,No,35.0,6.8
3,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$329.99,4.5,433,75,1000,5.0,FreeSync (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,39.0,13.1
4,https://www.bestbuy.com/site/hp-24-ips-led-fhd...,"HP - 24"" IPS LED FHD FreeSync Monitor (HDMI, V...",HP,$149.99,4.7,629,70,10000000,5.0,FreeSync (AMD Adaptive Sync),...,No,NaN,No,NaN,No,Yes,No,No,NaN,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet,240,12000000,1.0,FreeSync (AMD Adaptive Sync),...,NaN,NaN,NaN,NaN,Yes,Yes,No,No,NaN,13.6
547,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet,240,12000000,1.0,FreeSync (AMD Adaptive Sync),...,NaN,NaN,NaN,NaN,Yes,Yes,No,No,NaN,13.6
548,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 27"" IPS ...",HP,$249.99,Yet,Yet,75,10000000,5.0,FreeSync (AMD Adaptive Sync),...,No,NaN,No,NaN,No,Yes,No,NaN,27.0,8.2
549,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet,60,1300,4.0,FreeSync (AMD Adaptive Sync),...,No,NaN,Yes,Not Applicable,Yes,Yes,Yes,No,28.0,14.1


In [126]:
monitor_final=monitor.append(monitor_1)

In [127]:
monitor_final

,product_link,product_name,brand,current_price,rating,review_count,Refresh_Rate,Contrast_Ratio,Response_Time,Synchronization_Technology,...,Touch_Screen,Quantum_Dot_Technology,Headphone_Jack,Voice_Assistant,Wall_Mountable,Anti_Glare,Speaker,Webcam,Power_Consumption,Weight
0,https://www.bestbuy.com/site/samsung-t350-seri...,"Samsung - T350 Series 24"" IPS LED FHD, FreeSyn...",Samsung,$139.99,4.6,1838,75,1000,5.0,FreeSync (AMD Adaptive Sync),...,No,No,No,No,Yes,NaN,No,No,35.0,5.95
1,https://www.bestbuy.com/site/samsung-390-serie...,"Samsung - 390 Series 24"" LED Curved FHD FreeSy...",Samsung,$139.99,4.7,5106,60,3000,4.0,FreeSync (AMD Adaptive Sync),...,No,NaN,Yes,No,Yes,No,No,No,25.0,7.30
2,https://www.bestbuy.com/site/lg-34-ultrawide-f...,LG 34” UltraWide Full HD HDR Monitor with Free...,LG,$299.99,4.5,425,75,1000,5.0,FreeSync (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,39.0,13.10
3,https://www.bestbuy.com/site/dell-s3222dgm-32-...,"Dell - S3222DGM 32"" LED Curved QHD FreeSync Ga...",Dell,$299.99,4.8,277,165,3000,2.0,FreeSync Premium (AMD Adaptive Sync),...,No,No,Yes,Not Applicable,Yes,Yes,No,No,19.9,22.40
4,https://www.bestbuy.com/site/samsung-ur55-seri...,"Samsung - UR55 Series 28"" IPS 4K UHD Monitor -...",Samsung,$319.99,4.6,1840,60,1000,4.0,FreeSync (AMD Adaptive Sync),...,No,No,No,No,Yes,NaN,No,No,40.0,12.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet,240,12000000,1.0,FreeSync (AMD Adaptive Sync),...,NaN,NaN,NaN,NaN,Yes,Yes,No,No,NaN,13.60
547,https://www.bestbuy.com/site/geek-squad-certif...,Geek Squad Certified Refurbished OMEN X by HP ...,Geek,$427.99,Yet,Yet,240,12000000,1.0,FreeSync (AMD Adaptive Sync),...,NaN,NaN,NaN,NaN,Yes,Yes,No,No,NaN,13.60
548,https://www.bestbuy.com/site/hp-geek-squad-cer...,"HP - Geek Squad Certified Refurbished 27"" IPS ...",HP,$249.99,Yet,Yet,75,10000000,5.0,FreeSync (AMD Adaptive Sync),...,No,NaN,No,NaN,No,Yes,No,NaN,27.0,8.20
549,https://www.bestbuy.com/site/dell-geek-squad-c...,"Dell - Geek Squad Certified Refurbished 27"" IP...",Dell,$278.99,Yet,Yet,60,1300,4.0,FreeSync (AMD Adaptive Sync),...,No,NaN,Yes,Not Applicable,Yes,Yes,Yes,No,28.0,14.10


In [128]:
# Save the data to local   
monitor_final.to_csv('monitor_final.csv',index=False)  

In [131]:
monitor_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1102 entries, 0 to 550
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_link                1102 non-null   object 
 1   product_name                1102 non-null   object 
 2   brand                       1102 non-null   object 
 3   current_price               1102 non-null   object 
 4   rating                      1102 non-null   object 
 5   review_count                1102 non-null   object 
 6   Refresh_Rate                1094 non-null   object 
 7   Contrast_Ratio              1094 non-null   object 
 8   Response_Time               983 non-null    float64
 9   Synchronization_Technology  1056 non-null   object 
 10  Panel_Type                  1088 non-null   object 
 11  Curved_Screen               1069 non-null   object 
 12  Pixel_Density               184 non-null    float64
 13  Brightness                  1094 n